In [ ]:
import numpy as np
import scrublet as scr
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
from collections import Counter
import anndata as ann
import bbknn
import cellrank as cr
import scanpy as sc
import scvelo as scv
import loompy
import dynamo as dyn
dyn.session_info()

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

In [ ]:
adata = sc.read_h5ad("./h5ad/DNT_velocity_input.h5ad")

In [ ]:
preprocessor = dyn.pp.Preprocessor(cell_cycle_score_enable=True)
preprocessor.config_monocle_recipe(adata)
preprocessor.filter_cells_by_outliers_kwargs["keep_filtered"] = False
preprocessor.filter_genes_by_outliers_kwargs["inplace"] = True
preprocessor.preprocess_adata_monocle(adata)

In [ ]:
dyn.tl.dynamics(adata, model='stochastic', cores=48)

In [ ]:
dyn.tl.reduceDimension(adata)

In [ ]:
dyn.pl.umap(adata, color='DNT_C')

In [ ]:
dyn.tl.cell_velocities(adata, 
                       method='pearson', 
                       other_kernels_dict={'transform': 'sqrt'})

In [ ]:
dyn.tl.cell_wise_confidence(adata)

In [ ]:
dyn.vf.VectorField(adata, basis='umap', M=1000, pot_curl_div=True)

In [ ]:
dyn.pl.plot_energy(adata, basis='umap')

In [ ]:
dyn.pl.streamline_plot(adata, color=['DNT_C'], 
                       # basis='diffmap_', 
                       basis='umap', 
                       color_key = {
             'T4_CXCL13+PDCD1+': '#f0b98d',
             'T5_FOXP3+IL2RA+': '#ef9708',
             'T2_LEF1+CCR7+': '#E17B84',
             'T6_CXCL1+CXCL8+': '#b5bbe3',
             'T1_RGS1+CREM+': '#d33f6a',
             'T3_CD40LG+CCR6+': '#8dd593',

            'T8_IFNG+': '#023fa5',
             'T12_TRDV2+': '#BFC1D3', 
             'T7_PTGDS+': '#4a6fe3',
             'T9_CX3CR1+': '#c6dec7',
             'T13_TRDV1+': '#AC9EA0',
             'T11_ITGA1+': '#BEF197',
             'T14_HLA-DR+': '#00c0b9',
             'T10_GZMK+': '#9cded6',
            },
           show_arrowed_spines=True,
            # xy_grid_nums=(100,100),
            xy_grid_nums=(60,60),
            # xy_grid_nums=(90,90),
            method = 'gaussian',
            # method = 'SparseVFC',
            # density=1.1,
            # linewidth = 0.9,
            density=0.8,
            linewidth = 1.1, 
            # streamline_alpha = 0.9,
            show_legend=None,
           # show_legend='on data',
           # save_show_or_return = "save",
           #  save_kwargs = {"path": None, "prefix": "streamline_plot", "dpi": None, "ext": "png", "transparent": True, "close": True, "verbose": True},
           )